In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

In [2]:
df = pd.read_parquet("../Proyecto-Taxis-NYC/data/yellow_t_2018-01.parquet", engine="pyarrow")

In [4]:
#esto se uso inicilamente para poder darnos una idea de los datos a tratar

"""
def get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):
    labels,values,percent = list(),list(),list()
    if df.isna().sum().sum()>0:
        for column in df.columns:
            if df[column].isna().sum():
                labels.append(column)
                values.append(df[column].isna().sum())
                percent.append((df[column].isna().sum() / df.shape[0]) * 100)
        #Make a dataframe 
        missings=pd.DataFrame({'Colúmnas':labels,'Valores faltantes':values,'PorcentajePerdido':percent}).sort_values(by='PorcentajePerdido',ascending=False)
        

        if plot:
            plt.figure(figsize=figsize)
            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title('Porcentaje de valores perdidos',size=22)
            for i in plot.patches:
                plot.annotate(i.get_height(),
                (i.get_x() + i.get_width() / 2, i.get_height()),
                ha = 'center', va = 'baseline', fontsize = 12,
                color = 'black', xytext = (0,1),
                textcoords = 'offset pixels')
            locs, labels = plt.xticks()
            plt.setp(labels, rotation=rotation,size=16)
            plt.xlabel('Colúmnas',size=18)
            plt.ylabel('Porcentaje',size=18)
            plot.set_title('Porcentaje de valores perdidos',size=18)

        return missings
    else:
        return False
        """

#get_missings(df)

'\ndef get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):\n    labels,values,percent = list(),list(),list()\n    if df.isna().sum().sum()>0:\n        for column in df.columns:\n            if df[column].isna().sum():\n                labels.append(column)\n                values.append(df[column].isna().sum())\n                percent.append((df[column].isna().sum() / df.shape[0]) * 100)\n        #Make a dataframe \n        missings=pd.DataFrame({\'Colúmnas\':labels,\'Valores faltantes\':values,\'PorcentajePerdido\':percent}).sort_values(by=\'PorcentajePerdido\',ascending=False)\n        \n\n        if plot:\n            plt.figure(figsize=figsize)\n            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title(\'Porcentaje de valores perdidos\',size=22)\n            for i in plot.patches:\n                plot.annotate(i.get_height(),\n                (i.get_x() + i.get_width() / 2, i.get_height()),\n          

Aplicamos la transformacion de los tipos en caso de que vengan con uno incorrecto

In [11]:
df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day

Creamos una mascara del dataframe cada 3 dias

In [ ]:
df_Impar = df[df.dayofmonth % 3 == 0]
#df_Impar = df_Impar.copy()

Eliminamos las columnas que consideramos insignificantes

In [6]:
df_Impar.drop(columns=["congestion_surcharge","airport_fee","VendorID","store_and_fwd_flag"], inplace=True)

Sorteamos los value por date y filtramos los que estan fuera de Enero

In [9]:


def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

In [12]:
df_Impar.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
644899,2,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,N,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3
599145,2,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,N,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3
480025,1,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,N,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3
478103,1,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,N,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3
469218,2,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,N,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3


De ahora en mas df_Impar es el df a usar

In [14]:
df_Impar.shape

(2908992, 18)

Vamos a obtener los Boroughs de salida y de llegada.

In [16]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

In [17]:
df_Impar.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan


Trabajamos con el clima

In [18]:
clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/new york 2018-01-01 to 2018-02-01.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]

In [ ]:
def control_clima(dat):
    date = dat
    if clima["datetime"] >= date:
        clima = clima[clima["datetime"] >= date]
        date_next= date + datetime.timedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= datetime.timedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1))

In [19]:
clima.head()

,datetime,temp,feelslike,precip,snow,snowdepth,conditions
0,2018-01-01 00:00:00,-12.6,-20.8,0.0,0.0,0.68,Clear
1,2018-01-01 01:00:00,-12.6,-20.9,0.0,0.0,0.60,Clear
2,2018-01-01 02:00:00,-12.7,-21.1,0.0,0.0,0.51,Clear
3,2018-01-01 03:00:00,-13.2,-21.2,0.0,0.0,0.43,Clear
4,2018-01-01 04:00:00,-13.3,-20.9,0.0,0.0,0.34,Clear


creamos columna auxiliar de hora

In [20]:
df_Impar["hora"]= df_Impar.tpep_pickup_datetime.dt.floor("H")
clima.datetime = clima['datetime'].apply(pd.to_datetime)
clima["hora"]= clima['datetime'].dt.floor("H")

In [21]:
#cambiamos el nombre para poder hacer el merge
#zona.rename(columns={"datetime": "tpep_pickup_datetime"},inplace=True)

In [22]:
df_Impar.shape

(2908992, 19)

se cargaron los datos del clima al dataframe.

In [23]:
df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

In [24]:
dfsinceros=df_Impar[df_Impar.fare_amount != 0]
dfsinceros=dfsinceros[dfsinceros.fare_amount>0]
dfsinceros = dfsinceros[dfsinceros.trip_distance != 0]

In [25]:
df1=dfsinceros[dfsinceros.RatecodeID == 1]
df2=dfsinceros[dfsinceros.RatecodeID == 2]
df3=dfsinceros[dfsinceros.RatecodeID == 3]
df4=dfsinceros[dfsinceros.RatecodeID == 4]
df5=dfsinceros[dfsinceros.RatecodeID == 5]

In [26]:
def millas(df):
  df['km_dollar'] = df['trip_distance']/df['fare_amount']
  return df
  

In [27]:
df1 = millas(df1)
df2 = millas(df2)
df3 = millas(df3)
df4 = millas(df4)
df5 = millas(df5)

In [28]:
avg_km1 =df1['km_dollar'].mean()
avg_km2 =df2['km_dollar'].mean()
avg_km3 =df3['km_dollar'].mean()
avg_km4 =df4['km_dollar'].mean()
avg_km5 =df5['km_dollar'].mean()

In [29]:
df_Impar.shape

(2908992, 26)

se borran los que tienen cero en trip distance y fare amount

In [30]:

df_Impar["borrar"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]==0  and row["trip_distance"]==0 else 0, axis=1)
df_Impar = df_Impar.drop(df_Impar[df_Impar.borrar == 1].index)
df_Impar = df_Impar.drop(columns="borrar")

In [31]:
df_Impar.shape

(2908638, 26)

In [32]:
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km1 if row["RatecodeID"]==1  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km2 if row["RatecodeID"]==2  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km3 if row["RatecodeID"]==3  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km4 if row["RatecodeID"]==4  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km5 if row["RatecodeID"]==5  and row["trip_distance"]==0 else row["trip_distance"], axis=1)


Hasta ahora, se borraron las filas con trip distance y fare amount en cero, y se imputo la distancia recorrida en los campos correspondientes segun su Ratecode ID. restaria borrar los Ratecode ID 6 y 99 que son practicamente irrelevantes.

vamos a revisar y decidir que hacer con los precios negativos

Generamos csv para tablas sql

In [33]:
#df_Impar.to_csv('df_ImparSQL.csv', sep=',')

In [39]:
df_Impar[df_Impar.fare_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [40]:
df_Impar[df_Impar.trip_distance<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
11905,2018-01-03 05:24:43,2018-01-03 05:25:19,1,-732.619596,5,48,48,3,-34.8,0.0,-0.5,0.0,0.0,-0.3,-35.6,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
12080,2018-01-03 05:29:10,2018-01-03 05:29:53,1,-4.954313,3,129,129,4,-20.0,-0.5,0.0,0.0,0.0,-0.3,-20.8,3,Queens,Queens,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
13177,2018-01-03 05:49:31,2018-01-03 05:50:42,4,-0.614998,1,230,230,4,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
19874,2018-01-03 06:50:34,2018-01-03 06:50:42,2,-0.512498,1,132,132,4,-2.5,0.0,-0.5,0.0,0.0,-0.3,-3.3,3,Queens,Queens,2018-01-03 06:00:00,2018-01-03 06:00:00,-8.5,-8.5,0.0,0.0,0.08,Partially cloudy


In [38]:
df_Impar[df_Impar.total_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [26]:
df_Impar.loc[df_Impar['tpep_pickup_datetime'] == "2018-01-27 10:59:14"]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
2390960,2018-01-27 10:59:14,2018-01-27 11:20:14,1,1.00,1,234,186,1,13.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390961,2018-01-27 10:59:14,2018-01-27 11:07:50,1,1.70,1,236,151,1,8.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390962,2018-01-27 10:59:14,2018-01-27 11:21:12,1,4.35,1,236,246,2,18.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390963,2018-01-27 10:59:14,2018-01-27 11:21:08,1,3.34,1,148,246,1,16.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390964,2018-01-27 10:59:14,2018-01-27 11:02:01,1,0.40,1,262,263,1,4.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390965,2018-01-27 10:59:14,2018-01-27 11:10:44,1,2.77,1,48,43,2,11.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390966,2018-01-27 10:59:14,2018-01-27 11:06:16,1,1.61,1,237,170,1,7.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390967,2018-01-27 10:59:14,2018-01-27 11:01:22,1,0.77,1,237,236,2,4.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390968,2018-01-27 10:59:14,2018-01-27 11:09:15,3,1.01,1,90,68,2,8.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390969,2018-01-27 10:59:14,2018-01-27 11:08:39,1,1.50,99,238,142,1,8.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy


In [27]:
df_Impar.RatecodeID.unique()

array([ 1,  2,  5,  4,  3, 99,  6], dtype=int64)

In [28]:
df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)
#df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)

In [30]:
#el dataframe sin 0s
df_Impar['km_dollar'] = df_Impar['trip_distance']/df_Impar['fare_amount']

avg_km = df_Impar['km_dollar'].mean()

km_ride = df_Impar['trip_distance']


In [31]:
round(avg_km,4)

0.2345

In [32]:

price=df_Impar['fare_amount']

df_Impar['trip_distance']=df_Impar['trip_distance'].replace(to_replace = 0, value = df_Impar['km_dollar'].mean()*df_Impar['fare_amount'], inplace= True)


ValueError: Series.replace cannot use dict-value and non-None to_replace

In [ ]:
df_Impar.head(1)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03,2018-01-03,2,1.53,1,2,10.5,1.0,0.5,0.0,0.0,0.3,12.3,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714


In [91]:
dfRatecode=df_Impar.groupby("RatecodeID").agg({'trip_distance':'sum'})

In [97]:
df_count = df_Impar.RatecodeID.value_counts()


In [98]:
df_count

1     2835467
2       58056
5        8375
3        4710
4        1618
99         32
6          13
Name: RatecodeID, dtype: int64

In [ ]:

df_count = pd.DataFrame(df_count, columns=['RatecodeID'])


In [ ]:
df_count.reset_index(inplace=True)
df_count.rename(columns={"index": "RatecodeID","RatecodeID":"Cantidad"},inplace=True)


In [93]:
df_count.head(7)

,RatecodeID,Cantidad
0,1,2835467
1,2,58056
2,5,8375
3,3,4710
4,4,1618
5,99,32
6,6,13


In [94]:
dfRatecode = pd.merge(dfRatecode,df_count, how="left", on=["RatecodeID"])

In [96]:
dfRatecode.head()

,RatecodeID,trip_distance,Cantidad
0,1,7070382.33,2835467
1,2,987081.95,58056
2,3,77967.88,4710
3,4,30734.76,1618
4,5,49585.75,8375


In [95]:
dfRatecode["km_dollar"] = dfRatecode.trip_distance/dfRatecode.Cantidad

AttributeError: 'DataFrame' object has no attribute 'fare_amount'

In [ ]:
dfRatecode.head(10)

,RatecodeID,fare_amount,Cantidad,km_dollar
0,1,31479383.18,2835467,11.102010
1,2,3008696.50,58056,51.824041
2,3,310009.00,4710,65.819321
3,4,116236.45,1618,71.839586
4,5,529885.79,8375,63.269945
5,6,42.50,13,3.269231
6,99,1312.82,32,41.025625


In [ ]:
df_Impar.head(5)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.226190
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.163636
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.279245
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.261000


In [ ]:
type(avg_km)

numpy.float64

In [ ]:
df_Impar.head(5)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.226190
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.163636
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.279245
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.261000


In [ ]:
df_Impar.rename(columns={"tpep_pickup_datetime":"time_pickup","tpep_dropoff_datetime":"time_dropoff"},inplace=True)

In [ ]:
df_Impar.loc[df_Impar['trip_distance'] == 0]


,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,...,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions
85,2018-01-03 00:00:48,2018-01-03 00:00:53,1,0.0,5,1,15.0,0.0,0.5,3.16,...,18.96,3,Unknown,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
196,2018-01-03 00:01:49,2018-01-03 00:01:49,2,0.0,1,2,31.5,1.0,0.5,0.00,...,39.06,3,Queens,Unknown,-7.3,-13.5,0.0,0.0,0.33,Clear
343,2018-01-03 00:03:23,2018-01-03 00:03:26,1,0.0,5,1,5.0,0.0,0.5,0.00,...,5.80,3,Unknown,Brooklyn,-7.3,-13.5,0.0,0.0,0.33,Clear
354,2018-01-03 00:03:28,2018-01-04 00:03:02,2,0.0,5,1,20.0,0.0,0.0,0.00,...,20.30,3,Queens,Queens,-7.3,-13.5,0.0,0.0,0.33,Clear
516,2018-01-03 00:05:02,2018-01-03 00:05:31,1,0.0,5,1,69.2,0.0,0.0,0.00,...,69.50,3,Brooklyn,Brooklyn,-7.3,-13.5,0.0,0.0,0.33,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908613,2018-01-30 23:57:18,2018-01-30 23:57:44,1,0.0,1,3,2.5,0.5,0.5,0.00,...,3.80,30,Manhattan,Manhattan,-3.6,-8.9,0.0,0.0,0.91,Clear
2908666,2018-01-30 23:57:44,2018-01-30 23:57:48,1,0.0,1,3,2.5,0.5,0.5,0.00,...,3.80,30,Brooklyn,Brooklyn,-3.6,-8.9,0.0,0.0,0.91,Clear
2908754,2018-01-30 23:58:19,2018-01-30 23:58:19,1,0.0,1,2,2.5,0.5,0.5,0.00,...,3.80,30,Manhattan,Unknown,-3.6,-8.9,0.0,0.0,0.91,Clear
2908870,2018-01-30 23:59:06,2018-01-30 23:59:25,1,0.0,5,1,63.0,0.0,0.5,12.76,...,76.56,30,Manhattan,Manhattan,-3.6,-8.9,0.0,0.0,0.91,Clear


In [ ]:
pd.options.display.max_columns=None

In [ ]:
dfAux = df_Impar[df_Impar["trip_distance"] != 0]

yo sumaria todas las distancias de RatecodeID = 1, sumaria todas los precios de fare_amount, calcularia cuanto cuesta una milla.

In [ ]:
df_Impar.head(3)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions
0,2018-01-03,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.0,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
1,2018-01-03,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.0,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
2,2018-01-03,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.0,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear


In [ ]:
data_parcial["dayofmonth"] = data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
datafiltrada= data_parcial.head(0)



In [ ]:
datafiltrada

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth


In [ ]:
#data_parcial.loc[data_parcial.index == 148 ]

In [ ]:
#data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[0]

In [ ]:
data_parcial.head(2)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
0,722237,2,2018-01-03 23:03:00,2018-01-03 23:13:02,3,1.19,1,N,186,161,2,8.0,0.5,0.5,0.00,0.0,0.3,9.30,3
1,722390,2,2018-01-03 23:01:55,2018-01-03 23:09:39,1,1.13,1,N,234,186,1,7.0,0.5,0.5,2.08,0.0,0.3,10.38,3
